# Jupyter Notebook Interactive Mode Demo

### Importing necessary libraries

Import the required libraries needed for the RAG. Core libraries are dotenv, requests, httpx, pymilvus, and langchain. Langchain extensions are core, mistralai, milvus, community, huggingface, and text-splitters.

In [ ]:

# Installing dependencies if not already installed
!pip install os requests httpx pymilvus sqlite3
!pip install langchain langchain-core langchain-mistralai langchain-cohere langchain-milvus langchain-community langchain-text-splitters langchain-huggingface

import os
from dotenv import load_dotenv
from pymilvus import connections, utility
import sqlite3

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.schema import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_milvus import Milvus
from langchain_community.document_loaders import RecursiveUrlLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain_huggingface import HuggingFaceEmbeddings

from requests.exceptions import HTTPError
from httpx import HTTPStatusError
import warnings

warnings.filterwarnings('ignore')

print("Dependencies imported successfully.")

### Load env variables

Load the env variables needed for operation of the RAG. `CORPUS_SOURCE` can be changed to load a different corpus. `MISTRAL_API_KEY` contains the MistralAi API key. `MILVUS_URI` is the path for the milvus lite database file. `MODEL_NAME` is the embedding model used on the corpus.

In [ ]:
CORPUS_SOURCE = 'https://www.csusb.edu/its'
MISTRAL_API_KEY = os.environ.get("MISTRAL_API_KEY")
MILVUS_URI = "milvus/jupyter_milvus_vector.db"
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

print("ENV variables defined.")

### Function to create Vector Store (Milvus database)

Creates the `/milvus` directory if it doesn't already exist. Attempts to connect to the database file. Return a boolean based on whether the database exists.

In [ ]:
def vector_store_check(uri):
    """
    Returns response on whether the milvus database exists

    Returns:
        boolean
    """
    # Create the directory if it does not exist
    head = os.path.split(uri)
    os.makedirs(head[0], exist_ok=True)
    
    # Connect to the Milvus database
    connections.connect("default",uri=uri)

    # Return True if exists, False otherwise
    return utility.has_collection("IT_support")

print("Function `vector_store_check` defined.")

### Function to retrieve embedding function

Gets to embedding function from HuggingFace based on the `MODEL_NAME` ENV variable. Returns the embedding model

In [ ]:
def get_embedding_function():
    """
    Returns embedding function for the model

    Returns:
        embedding function
    """
    embedding_function = HuggingFaceEmbeddings(model_name=MODEL_NAME)

    print("Embedding function loaded.")
    return embedding_function

print("Function `get_embedding_function` defined.")

### Function to load documents from the web

Load documents from the web recursively based on `CORPUS_SOURCE`. Prevents loaded pages that are outside of the base_url of `CORPUS_SOURCE`. Returns the documents that are loaded.

In [ ]:
def load_documents_from_web():
    """
    Load the documents from the web and store the page contents

    Returns:
        list: The documents loaded from the web
    """
    loader = RecursiveUrlLoader(
        url=CORPUS_SOURCE,
        prevent_outside=True,
        base_url=CORPUS_SOURCE
        )
    documents = loader.load()
    
    print("Documents loaded.")
    return documents

print("Function `load_documents_from_web` defined.")

### Function to load existing vector store (Milvus database)

Takes the path to the database and embedding function and connects to the database. Returns to connected vector store.

In [ ]:
def load_existing_db(uri=MILVUS_URI):
    """
    Load an existing vector store from the local Milvus database specified by the URI.

    Args:
        uri (str, optional): Path to the local milvus db. Defaults to MILVUS_URI.

    Returns:
        vector_store: The vector store created
    """
    # Load an existing vector store
    vector_store = Milvus(
        collection_name="IT_support",
        embedding_function = get_embedding_function(),
        connection_args={"uri": uri},
    )
    
    print("Vector store loaded.")
    return vector_store

print("Function `load_existing_db` defined.")

### Function to split documents

Takes the documents loaded from `load_documents_from_web` and splits them into chunks of 1000 characters. Overlaps 300 characters to ensure no context is missing. Returns the documents split into chunks.

In [ ]:
def split_documents(documents):
    """
    Split the documents into chunks

    Args:
        documents (list): The documents to split

    Returns:
        list: list of chunks of documents
    """
    # Create a text splitter to split the documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=300,
        is_separator_regex=False,
    )
    
    docs = text_splitter.split_documents(documents)
    
    print("Documents successfully split.")
    return docs

print("Function `split_documents` defined.")

### Function to create vector store (Milvus database)

Takes the documents from `load_documents_from_web`, embedding function from `get_embedding_function`, and database path to create the vector store. Once it is created it returns the created vector store.

In [ ]:
def create_vector_store(docs, embeddings, uri):
    """
    This function initializes a vector store using the provided documents and embeddings.

    Args:
        docs (list): A list of documents to be stored in the vector store.
        embeddings : A function or model that generates embeddings for the documents.
        uri (str): Path to the local milvus db

    Returns:
        vector_store: The vector store created
    """
    # Create a new vector store and drop any existing one
    vector_store = Milvus.from_documents(
        documents=docs,
        embedding=embeddings,
        collection_name="IT_support",
        connection_args={"uri": uri},
        drop_old=True,
    )
    
    print("Vector store created.")
    return vector_store

print("Function `create_vector_store` defined.")

# Base function to initialize Milvus

Main function in initializing Milvus. Uses the previous functions to fully create the vector store. Running `initialize_milvus` will call all other functions needed to create the vector store.

In [ ]:
def initialize_milvus(uri: str=MILVUS_URI):
    """
    Initialize the vector store for the RAG model

    Args:
        uri (str, optional): Path to the local milvus db. Defaults to MILVUS_URI.

    Returns:
        vector_store: The vector store created
    """
    if vector_store_check(uri):
        vector_store = load_existing_db(uri)
        print("Embeddings loaded from existing Database")
    else:
        embeddings = get_embedding_function()
        print("Embeddings Loaded")
        documents = load_documents_from_web()
        print("Documents Loaded")
        print(len(documents))
    
        # Split the documents into chunks
        docs = split_documents(documents=documents)
        print("Documents Splitting completed")
    
        vector_store = create_vector_store(docs, embeddings, uri)

    print("Milvus successfully initialized.")
    #return vector_store

print("Function `initialize_milvus` defined.")

### Initialize vector store (Milvus database)

Due to the embedding function, this will take a significant amount of time. Please be patient until it completes.

In [ ]:
print("Starting Milvus initialization.")
initialize_milvus()

### Function to create RAG prompt

Define the `PROMPT_TEMPLATE` and assign the roles through `ChatPromptTemplate`. `<context>` tags contain the document context. `<question>` tags contain the user question.

In [ ]:
def create_prompt():
    """
    Create a prompt template for the RAG model

    Returns:
        PromptTemplate: The prompt template for the RAG model
    """
    # Define the prompt template
    PROMPT_TEMPLATE = """\
    You are an AI assistant that provides answers strictly based on the provided context. Adhere to these guidelines:
     - Only answer questions based on the content within the <context> tags.
     - If the <context> does not contain information related to the question, respond only with: "I don't have enough information to answer this question."
     - For unclear questions or questions that lack specific context, request clarification from the user.
     - Provide specific, concise ansewrs. Where relevant information includes statistics or numbers, include them in the response.
     - Avoid adding any information, assumption, or external knowledge. Answer accurately within the scope of the given context and do not guess.
     - If information is missing, respond only with: "I don't have enough information to answer this question."
    """

    prompt = ChatPromptTemplate.from_messages([
        ("system", PROMPT_TEMPLATE),
        ("human", "<question>{input}</question>\n\n<context>{context}</context>"),
    ])

    print("Prompt template defined.")
    return prompt

print("Function `create_prompt` defined.")

### Function to query RAG model

Loads the MistralAI model, prompt template, and vector store (Milvus database). Converts the vector store into a retriever, which will retrieve the documents containing context. Create a document chain containing all the documents with context. Create a retrieval_chain that uses the documents and retriever to load context based on user question. Based on the retrieved context documents, retrieve source metadata.

In [ ]:
def query_rag(query):
    """
    Entry point for the RAG model to generate an answer to a given query

    This function initializes the RAG model, sets up the necessary components such as the prompt template, vector store, 
    retriever, document chain, and retrieval chain, and then generates a response to the provided query.

    Args:
        query (str): The query string for which an answer is to be generated.
    
    Returns:
        str: The answer to the query
    """
    # Define the model
    model = ChatMistralAI(model='open-mistral-7b')
    print("Model Loaded")

    prompt = create_prompt()

    # Load the vector store and create the retriever
    vector_store = load_existing_db(uri=MILVUS_URI)
    retriever = vector_store.as_retriever()
    try:
        document_chain = create_stuff_documents_chain(model, prompt)
        print("Document Chain Created")

        retrieval_chain = create_retrieval_chain(retriever, document_chain)
        print("Retrieval Chain Created")
    
        # Generate a response to the query
        response = retrieval_chain.invoke({"input": f"{query}"})
    except HTTPStatusError as e:
        print(f"HTTPStatusError: {e}")
        if e.response.status_code == 429:
            return "I am currently experiencing high traffic. Please try again later.", []
        return "I am unable to answer this question at the moment. Please try again later.", []
    
    # logic to add sources to the response
    max_relevant_sources = 4 # number of sources at most to be added to the response
    all_sources = ""
    sources = []
    count = 1
    for i in range(max_relevant_sources):
        try:
            source = response["context"][i].metadata["source"]
            # check if the source is already added to the list
            if source not in sources:
                sources.append(source)
                all_sources += f"[Source {count}]({source}), "
                count += 1
        except IndexError: # if there are no more sources to add
            break
    all_sources = all_sources[:-2] # remove the last comma and space
    response["answer"] += f"\n\nSources: {all_sources}"
    print("------------------------------------------------------------------------")
    print("Response Generated:\n")
    
    return response["answer"]

print("Function `query_rag` defined.")

### Get response from RAG

Send RAG a user question and get the RAG response. Print the response.

In [ ]:
response = query_rag("how do I connect to the wifi?")

print(response)